<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/logo.png"/>
<img align="right" src="images/tf.png"/>


# Tutorial

This notebook gets you started with coding in the
Peshitta, the Syriac Old Testament,
using [Text-Fabric](https://github.com/Dans-labs/text-fabric)

Chances are that a bit of reading about the underlying
[data model](https://github.com/Dans-labs/text-fabric/wiki/Data-model)
helps you to follow the exercises below, and vice versa.

Most programs start with loading a few modules.
In the next cell, the first line loads standard modules that come with Python itself,
and the second cell loads Text-Fabric.

Before you can run this, you need to install it.
The basic instruction for that is, on a terminal:

```
pip install text-fabric
```

if you have installed Python with the help of Anaconda, or

```
sudo -H pip3 install text-fabric
```
if you have installed Python from [python.org](https://www.python.org).

Make sure that you do all this with Python **3**, not 2.

In [1]:
from tf.extra.peshitta import Peshitta

# Load Features
The data of the corpus is organized in features.
They are *columns* of data.
Think of the text as a gigantic spreadsheet, where row 1 corresponds to the
first word, row 2 to the second word, and so on, for all 400,000+ words.

The letters of each word is a column `form` in that spreadsheet.

The corpus contains ca. 10 columns, not only for the words, but also for 
textual objects, such as *books*, *chapters*, and *verses*.

Instead of putting that information in one big table, the data is organized in separate columns.
We call those columns **features**.

We just load the features we need for this tutorial.
Later on, where we use them, it will become clear what they mean.

In [2]:
P = Peshitta(hoist=globals())

Using peshitta-0.1 r0.1 in ~/text-fabric-data/etcbc/peshitta/tf/0.1


**Documentation:** <a target="_blank" href="https://github.com/etcbc/peshitta/blob/master/docs/transcription" title="{provenance of this corpus}">PESHITTA</a> <a target="_blank" href="https://github.com/etcbc/peshitta/blob/master/docs/transcription-0.1.md" title="PESHITTA feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Peshitta/" title="Peshitta API documentation">Peshitta API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 6.1.0</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>


This notebook online:
<a target="_blank" href="https://nbviewer.jupyter.org/github/etcbc/peshitta/blob/master/tutorial/start.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/etcbc/peshitta/blob/master/tutorial/start.ipynb">GitHub</a>


The result of this all is that we have a bunch of special variables at our disposal
that give us access to the text and data of the Hebrew Bible.

In [57]:
for (section, newNames) in names:
  print(f'{section:<20}: {", ".join(newNames)}')

computed-data       : C Computed, Call AllComputeds, Cs ComputedString
edge-features       : E Edge, Eall AllEdges, Es EdgeString
loading             : TF, ensureLoaded, ignored, loadLog
locality            : L Locality
messaging           : cache, error, indent, info, reset
navigating-nodes    : N Nodes, sortKey, otypeRank, sortNodes
node-features       : F Feature, Fall AllFeatures, Fs FeatureString
searching           : S Search
text                : T Text


At this point it is helpful to throw a quick glance at the text-fabric
[API documentation](https://github.com/Dans-labs/text-fabric/wiki/Api)
especially the right side bar.

The most essential thing for now is that we can use `F` to access the data in the features
we've loaded.
But there is more, such as `N`, which helps us to walk over the text, as we see in a minute.

# Counting

In order to get acquainted with the data, we start with the simple task of counting.

## Count all nodes
We use the 
[`N()` generator](https://github.com/Dans-labs/text-fabric/wiki/Api#walking-through-nodes)
to walk through the nodes.

We compared corpus to a gigantic spreadsheet, where the rows correspond to the words.
In Text-Fabric, we call the rows `slots`, because they are the textual positions that can be filled with words.

We also mentioned that there are also more textual objects. 
They are the verses, chapters and books.
They also correspond to rows in the big spreadsheet.

In Text-Fabric we call all these rows *nodes*, and the `N()` generator
carries us through those nodes in the textual order.

Just one extra thing: the `info` statements generate timed messages.
If you use them instead of `print` you'll get a sense of the amount of time that 
the various processing steps typically need.

In [31]:
indent(reset=True)
info('Counting nodes ...')

i = 0
for n in N(): i += 1

info('{} nodes'.format(i))

  0.00s Counting nodes ...
  0.09s 458807 nodes


## What are those nodes?
Every node has a type, like word, or phrase, sentence.
We know that we have approximately 100,000 words and a few other nodes.
But what exactly are they?

Text-Fabric has two special features, `otype` and `oslots`, that must occur in every Text-Fabric data set.
`otype` tells you for each node its type, and you can ask for the number of `slot`s in the text.

Here we go!

In [32]:
F.otype.slotType

'word'

In [33]:
F.otype.maxSlot

426163

In [34]:
F.otype.maxNode

458807

In [35]:
F.otype.all

('book', 'chapter', 'verse', 'word')

In [36]:
C.levels.data

(('book', 6556.3538461538465, 426164, 426228),
 ('chapter', 336.09069400630915, 426229, 427496),
 ('verse', 13.610648015074574, 427497, 458807),
 ('word', 1, 1, 426163))

This is interesting: above you see all the textual objects, with the average size of their objects,
the node where they start, and the node where they end.

## Count individual object types
This is an intuitive way to count the number of nodes in each type.
Note in passing, how we use the `indent` in conjunction with `info` to produce neat timed 
and indented progress messages.

In [37]:
indent(reset=True)
info('counting objects ...')

for otype in F.otype.all:
    i = 0
    indent(level=1, reset=True)

    for n in F.otype.s(otype): i+=1

    info('{:>7} {}s'.format(i, otype))

indent(level=0)
info('Done')

  0.00s counting objects ...
   |     0.00s      65 books
   |     0.00s    1268 chapters
   |     0.01s   31311 verses
   |     0.05s  426163 words
  0.07s Done


# Feature statistics

`F`
gives access to all features.
Every feature has a method
`freqList()`
to generate a frequency list of its values, higher frequencies first.
Here are the words in etcbc transliteration, only the top 10:

In [38]:
F.word_etcbc.freqList()[0:10]

(('MN', 8915),
 ('<L', 5316),
 ('MRJ>', 4572),
 ('L>', 4449),
 ('MVL', 3963),
 ('WL>', 3209),
 ('>JK', 3199),
 ('LH', 3124),
 ('>NWN', 2620),
 ('HW>', 2476))

# Layer API
We travel upwards and downwards, forwards and backwards through the nodes.
The Layer-API (`L`) provides functions: `u()` for going up, and `d()` for going down,
`n()` for going to next nodes and `p()` for going to previous nodes.

These directions are indirect notions: nodes are just numbers, but by means of the
`oslots` feature they are linked to slots. One node *contains* an other node, if the one is linked to a set of slots that contains the set of slots that the other is linked to.
And one if next or previous to an other, if its slots follow of precede the slots of the other one.

`L.u(node)` **Up** is going to nodes that embed `node`.

`L.d(node)` **Down** is the opposite direction, to those that are contained in `node`.

`L.n(node)` **Next** are the next *adjacent* nodes, i.e. nodes whose first slot comes immediately after the last slot of `node`.

`L.p(node)` **Previous** are the previous *adjacent* nodes, i.e. nodes whose last slot comes immediately before the first slot of `node`.

All these functions yield nodes of all possible otypes.
By passing an optional parameter, you can restrict the results to nodes of that type.

The result are ordered according to the order of things in the text.

The functions return always a tuple, even if there is just one node in the result.

## Going up
We go from the first word to the book it contains.
Note the `[0]` at the end. You expect one book, yet `L` returns a tuple. 
To get the only element of that tuple, you need to do that `[0]`.

If you are like me, you keep forgetting it, and that will lead to weird error messages later on.

In [39]:
firstBook = L.u(1, otype='book')[0]
print(firstBook)

426164


And let's see all the containing objects of word 3:

In [40]:
w = 3
for otype in F.otype.all:
    if otype == F.otype.slotType: continue
    up = L.u(w, otype=otype)
    upNode = 'x' if len(up) == 0 else up[0]
    print('word {} is contained in {} {}'.format(w, otype, upNode))

word 3 is contained in book 426164
word 3 is contained in chapter 426229
word 3 is contained in verse 427497


## Going next
Let's go to the next nodes of the first book.

In [41]:
afterFirstBook = L.n(firstBook)
for n in afterFirstBook:
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))
secondBook = L.n(firstBook, otype='book')[0]

  20071: word          first slot=20071 , last slot=20071 
 429029: verse         first slot=20071 , last slot=20081 
 426279: chapter       first slot=20071 , last slot=20325 
 426165: book          first slot=20071 , last slot=36392 


## Going previous

And let's see what is right before the second book.

In [42]:
for n in L.p(secondBook):
    print('{:>7}: {:<13} first slot={:<6}, last slot={:<6}'.format(
        n, F.otype.v(n),
        E.oslots.s(n)[0],
        E.oslots.s(n)[-1],
    ))

 426164: book          first slot=1     , last slot=20070 
 426278: chapter       first slot=19735 , last slot=20070 
 429028: verse         first slot=20058 , last slot=20070 
  20070: word          first slot=20070 , last slot=20070 


## Going down

We go to the chapters of the second book, and just count them.

In [43]:
chapters = L.d(secondBook, otype='chapter')
print(len(chapters))

40


## The first verse
We pick the first verse and the first word, and explore what is above and below them.

In [44]:
for n in [1, L.u(1, otype='verse')[0]]:
    indent(level=0)
    info('Node {}'.format(n), tm=False)
    indent(level=1)
    info('UP', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.u(n)]), tm=False)
    indent(level=1)
    info('DOWN', tm=False)
    indent(level=2)
    info('\n'.join(['{:<15} {}'.format(u, F.otype.v(u)) for u in L.d(n)]), tm=False)
indent(level=0)
info('Done', tm=False)

Node 1
   |   UP
   |      |   427497          verse
   |      |   426229          chapter
   |      |   426164          book
   |   DOWN
   |      |   
Node 427497
   |   UP
   |      |   426229          chapter
   |      |   426164          book
   |   DOWN
   |      |   1               word
   |      |   2               word
   |      |   3               word
   |      |   4               word
   |      |   5               word
   |      |   6               word
   |      |   7               word
Done


# Text API

So far, we have mainly seen nodes and their numbers, and the names of node types.
You would almost forget that we are dealing with text.
So let's try to see some text.

In the same way as `F` gives access to feature data,
`T` gives access to the text.
That is also feature data, but you can tell Text-Fabric which features are specifically
carrying the text, and in return Text-Fabric offers you
a Text API: `T`.

## Formats
Syriac text can be represented in a number of ways:

* in transliteration, or in Syriac characters,
* showing the actual text or only the lexemes,

If you wonder where the information about text formats is stored: 
not in the program text-fabric, but in the data set.
It has a feature `otext`, which specifies the formats and which features
must be used to produce them. `otext` is the third special feature in a TF data set,
next to `otype` and `oslots`. 
It is an optional feature. 
If it is absent, there will be no `T` API.

Here is a list of all available formats in this data set.

In [45]:
sorted(T.formats)

['text-orig-full', 'text-trans-full']

## Using the formats
Now let's use those formats to print out the first verse of the Hebrew Bible.

In [54]:
for fmt in sorted(T.formats):
    print('{}:\n\t{}'.format(fmt, T.text(range(1,12), fmt=fmt)))

text-orig-full:
	ܒܪܫܝܬ ܒܪܐ ܐܠܗܐ. ܝܬ ܫܡܝܐ ܘܝܬ ܐܪܥܐ. ܐܪܥܐ ܗܘܬ ܬܘܗ ܘܒܘܗ̇. 
text-trans-full:
	BRCJT BR> >LH>=. JT CMJ> WJT >R<>=. >R<> HWT TWH WBWH^=. 


If we do not specify a format, the **default** format is used (`text-orig-full`).

In [47]:
print(T.text(range(1,12)))

ܒܪܫܝܬ ܒܪܐ ܐܠܗܐ. ܝܬ ܫܡܝܐ ܘܝܬ ܐܪܥܐ. ܐܪܥܐ ܗܘܬ ܬܘܗ ܘܒܘܗ̇. 


## Whole text in all formats in less than a second
Part of the pleasure of working with computers is that they can crunch massive amounts of data.
The text of the Hebrew Bible is a piece of cake.

It takes just ten seconds to have that cake and eat it. 
In nearly a dozen formats.

In [48]:
indent(reset=True)
info('writing plain text of whole Syriac New Testament in all formats')

text = collections.defaultdict(list)

for v in F.otype.s('verse'):
    words = L.d(v, 'word')
    for fmt in sorted(T.formats):
        text[fmt].append(T.text(words, fmt=fmt))

info('done {} formats'.format(len(text)))

for fmt in sorted(text):
    print('{}\n{}\n'.format(fmt, '\n'.join(text[fmt][0:5])))

  0.00s writing plain text of whole Syriac New Testament in all formats
  1.81s done 2 formats
text-orig-full
ܒܪܫܝܬ ܒܪܐ ܐܠܗܐ. ܝܬ ܫܡܝܐ ܘܝܬ ܐܪܥܐ. 
ܐܪܥܐ ܗܘܬ ܬܘܗ ܘܒܘܗ̇. ܘܚܫܘܟܐ ܥܠ ܐ̈ܦܝ ܬܗܘܡܐ. ܘܪܘܚܗ ܕܐܠܗܐ ܡܪܚܦܐ ܥܠ ܐ̈ܦܝ ܡ̈ܝܐ. 
ܘܐܡ̣ܪ ܐܠܗܐ ܢܗܘܐ ܢܘܗܪܐ. ܘܗܘ̣ܐ ܢܘܗܪܐ. 
ܘܚ̣ܙܐ ܐܠܗܐ ܠܢܘܗܪܐ ܕܫܦܝܪ. ܘܦ̣ܪܫ ܐܠܗܐ ܒܝܬ ܢܘܗܪܐ ܠܚܫܘܟܐ. 
ܘܩ̣ܪܐ ܐܠܗܐ ܠܢܘܗܪܐ ܐܝܡܡܐ. ܘܠܚܫܘܟܐ ܩ̣ܪܐ ܠܠܝܐ. ܘܗܘ̣ܐ ܪܡܫܐ ܘܗܘܐ ܨܦܪܐ ܝܘܡܐ ܚܕ. 

text-trans-full
BRCJT BR> >LH>=. JT CMJ> WJT >R<>=. 
>R<> HWT TWH WBWH^=. WXCWK> <L >"PJ THWM>=. WRWXH D>LH> MRXP> <L >"PJ M"J>=. 
W>M#R >LH> NHW> NWHR>=. WHW#> NWHR>=. 
WX#Z> >LH> LNWHR> DCPJR=. WP#RC >LH> BJT NWHR> LXCWK>=. 
WQ#R> >LH> LNWHR> >JMM>=. WLXCWK> Q#R> LLJ>=. WHW#> RMC> WHW> YPR> JWM> XD=. 



### The full plain text
We write a few formats to file, in your `Downloads` folder.

In [49]:
orig = 'text-orig-full'
trans = 'text-trans-full'
for fmt in (orig, trans):
    with open(os.path.expanduser(f'~/Downloads/Peshitta-{fmt}.txt'), 'w') as f:
        f.write('\n'.join(text[fmt]))

In [50]:
!head -n 20 ~/Downloads/Peshitta-{orig}.txt

ܒܪܫܝܬ ܒܪܐ ܐܠܗܐ. ܝܬ ܫܡܝܐ ܘܝܬ ܐܪܥܐ. 
ܐܪܥܐ ܗܘܬ ܬܘܗ ܘܒܘܗ̇. ܘܚܫܘܟܐ ܥܠ ܐ̈ܦܝ ܬܗܘܡܐ. ܘܪܘܚܗ ܕܐܠܗܐ ܡܪܚܦܐ ܥܠ ܐ̈ܦܝ ܡ̈ܝܐ. 
ܘܐܡ̣ܪ ܐܠܗܐ ܢܗܘܐ ܢܘܗܪܐ. ܘܗܘ̣ܐ ܢܘܗܪܐ. 
ܘܚ̣ܙܐ ܐܠܗܐ ܠܢܘܗܪܐ ܕܫܦܝܪ. ܘܦ̣ܪܫ ܐܠܗܐ ܒܝܬ ܢܘܗܪܐ ܠܚܫܘܟܐ. 
ܘܩ̣ܪܐ ܐܠܗܐ ܠܢܘܗܪܐ ܐܝܡܡܐ. ܘܠܚܫܘܟܐ ܩ̣ܪܐ ܠܠܝܐ. ܘܗܘ̣ܐ ܪܡܫܐ ܘܗܘܐ ܨܦܪܐ ܝܘܡܐ ܚܕ. 
ܘܐܡ̣ܪ ܐܠܗܐ ܢܗܘܐ ܐܪܩܝܥܐ ܒܡܨܥܬ ܡ̈ܝܐ. ܘܢܗܘܐ ܦܪܫ̇ ܒܝܬ ܡ̈ܝܐ ܠܡ̈ܝܐ. 
ܘܥܒ̣ܕ ܐܠܗܐ ܐܪܩܝܥܐ. ܘܦ̣ܪܫ ܒܝܬ ܡ̈ܝܐ ܕܠܬܚܬ ܡܢ ܐܪܩܝܥܐ. ܘܒܝܬ ܡ̈ܝܐ ܕܠܥܠ ܡܢ ܐܪܩܝܥܐ. ܘܗܘ̣ܐ ܗܟܢܐ. 
ܘܩ̣ܪܐ ܐܠܗܐ ܠܐܪܩܝܥܐ ܫܡܝܐ. ܘܗܘ̣ܐ ܪܡܫܐ ܘܗܘܐ ܨܦܪܐ ܝܘܡܐ ܕܬܪ̈ܝܢ. 
ܘܐܡ̣ܪ ܐܠܗܐ ܢܬܟܢܫܘܢ ܡ̈ܝܐ ܕܠܬܚܬ ܡܢ ܫܡܝܐ ܠܐܬܪܐ ܚܕ ܘܬܬܚܙܐ ܝܒܝܫܬܐ. ܘܗܘ̣ܐ ܗܟܢܐ. 
ܘܩ̣ܪܐ ܐܠܗܐ ܠܝܒܝܫܬܐ ܐܪܥܐ. ܘܠܟܢܫܐ ܕܡ̈ܝܐ ܩ̣ܪܐ ܝܡ̈ܡܐ. ܘܚ̣ܙܐ ܐܠܗܐ ܕܫܦܝܪ. 
ܘܐܡ̣ܪ ܐܠܗܐ ܬܦܩ ܐܪܥܐ ܬܕܐܐ ܥܣܒܐ ܕܡܙܕܪܥ ܙܪܥܐ ܠܓܢܣܗ. ܘܐܝܠܢܐ ܕܦܐܪ̈ܐ ܕܥܒ̇ܕ ܦܐܪ̈ܐ ܠܓܢܣܗ ܕܢܨܒܬܗ ܒܗ ܥܠ ܐܪܥܐ. ܘܗ̣ܘܐ ܗܟܢܐ. 
ܘܐܦ̣ܩܬ ܐܪܥܐ ܬܕܐܐ. ܥܣܒܐ ܕܡܙܕܪܥ ܙܪܥܐ ܠܓܢܣܗ. ܘܐܝܠܢܐ ܕܥܒ̇ܕ ܦܐܪ̈ܐ ܕܢܨܒܬܗ ܒܗ ܠܓܢܣܗ. ܘܚ̣ܙܐ ܐܠܗܐ ܕܫܦܝܪ. 
ܘܗܘ̣ܐ ܪܡܫܐ ܘܗܘ̣ܐ ܨܦܪܐ. ܝܘܡܐ ܕܬܠܬܐ. 
ܘܐܡ̣ܪ ܐܠܗܐ. ܢܗܘܘܢ ܢܗܝܪ̈ܐ ܒܐܪܩܝܥܐ ܕܫܡܝܐ. ܠܡܦܪܫ ܒܝܬ ܐܝܡܡܐ ܠܠܠܝܐ  . ܘܢܗܘܘܢ ܠܐܬ̈ܘܬܐ ܘܠܙܒ̈ܢܐ. ܘܠܝ̈ܘܡܬܐ ܘܠܫ̈ܢܝܐ. 
ܘܢܗܘܘܢ ܡܢܗܪ̈

## Book names

For Bible book names, we can use several languages.
Well, in this case we have just English.

### Languages
Here are the languages that we can use for book names.
These languages come from the features `book@ll`, where `ll` is a two letter
ISO language code. Have a look in your data directory, you can't miss them.

In [51]:
T.languages

{'': {'language': 'default', 'languageEnglish': 'default'},
 'en': {'language': 'English', 'languageEnglish': 'English'}}

## Sections

A section is a book, a chapter or a verse.
Knowledge of sections is not baked into Text-Fabric. 
The config feature `otext.tf` may specify three section levels, and tell
what the corresponding node types and features are.

From that knowledge it can construct mappings from nodes to sections, e.g. from verse
nodes to tuples of the form:

    (bookName, chapterNumber, verseNumber)
   
Here are examples of getting the section that corresponds to a node and vice versa.

**NB:** `sectionFromNode` always delivers a verse specification, either from the
first slot belonging to that node, or, if `lastSlot`, from the last slot
belonging to that node.

In [52]:
for x in (
    ('section of first word',     T.sectionFromNode(1)                            ),
    ('node of Genesis 1:1',       T.nodeFromSection(('Genesis', 1, 1))            ),
    ('node of book Genesis',      T.nodeFromSection(('Genesis',))                 ),
    ('node of chapter Genesis 1', T.nodeFromSection(('Genesis', 1))               ),
    ('section of book node',      T.sectionFromNode(109641)                      ),
    ('idem, now last word',       T.sectionFromNode(109641, lastSlot=True)       ),
    ('section of chapter node',   T.sectionFromNode(109668)                      ),
    ('idem, now last word',       T.sectionFromNode(109668, lastSlot=True)       ),
): print('{:<30} {}'.format(*x))

section of first word          ('Genesis', 1, 1)
node of Genesis 1:1            427497
node of book Genesis           426164
node of chapter Genesis 1      426229
section of book node           ('1Samuel', 5, 4)
idem, now last word            ('1Samuel', 5, 4)
section of chapter node        ('1Samuel', 5, 5)
idem, now last word            ('1Samuel', 5, 5)


# Next steps

By now you have an impression how to compute around in the text.
While this is still the beginning, I hope you already sense the power of unlimited programmatic access
to all the bits and bytes in the data set.

Here are a few directions for unleashing that power.

## Search
Text-Fabric contains a flexible search engine, that does not only work for this data,
but also for data that you add to it.
There is a tutorial dedicated to [search](search.ipynb).
And if you already know MQL queries, you can build from that in
[searchFromMQL](searchFromMQL.ipynb).


## Add your own data
If you study the additional data, you can observe how that data is created and also
how it is turned into a text-fabric data module.
The last step is incredibly easy. You can write out every Python dictionary where the keys are numbers
and the values string or numbers as a Text-Fabric feature.
When you are creating data, you have already constructed those dictionaries, so writing
them out is just one method call.
See for example how the
[flowchart](https://github.com/ETCBC/valence/blob/master/programs/flowchart.ipynb#Add-sense-feature-to-valence-module)
notebook in valence writes out verb sense data.

You can then easily share your new features on GitHub, so that your colleagues everywhere 
can try it out for themselves.

## Export to Emdros MQL

[EMDROS](http://emdros.org), written by Ulrik Petersen,
is a text database system with the powerful *topographic* query language MQL.
The ideas are based on a model devised by Christ-Jan Doedens in
[Text Databases: One Database Model and Several Retrieval Languages](https://books.google.nl/books?id=9ggOBRz1dO4C).

Text-Fabric's model of slots, nodes and edges is a fairly straightforward translation of the models of Christ-Jan Doedens and Ulrik Petersen.

[SHEBANQ](https://shebanq.ancient-data.org) uses EMDROS to offer users to execute and save MQL queries against the Hebrew Text Database of the ETCBC.

So it is kind of logical and convenient to be able to work with a Text-Fabric resource through MQL.

If you have obtained an MQL dataset somehow, you can turn it into a text-fabric data set by `importMQL()`,
which we will not show here.

And if you want to export a Text-Fabric data set to MQL, that is also possible.

After the `Fabric(modules=...)` call, you can call `exportMQL()` in order to save all features of the
indicated modules into a big MQL dump, which can be imported by an EMDROS database.

In [53]:
TF.exportMQL('peshitta','~/Downloads')

  0.00s Checking features of dataset peshitta


   |     0.00s feature "book@en" => "book_en"


  0.01s 8 features to export to MQL ...
  0.02s Loading 8 features
  0.02s Writing enumerations
	book           :   65 values, 13 not a name, e.g. «1Chr»
	book_en        :   65 values, 13 not a name, e.g. «1Chronicles»
	trailer        :   14 values, 14 not a name, e.g. « »
	trailer_etcbc  :   14 values, 14 not a name, e.g. « »
  0.25s Mapping 8 features onto 4 object types
  0.54s Writing 8 features as data in 4 object types
   |     0.00s word data ...
   |      |     0.42s batch of size                6.3MB with   50000 of   50000 words
   |      |     0.93s batch of size                6.3MB with   50000 of  100000 words
   |      |     1.61s batch of size                6.4MB with   50000 of  150000 words
   |      |     2.16s batch of size                6.4MB with   50000 of  200000 words
   |      |     3.03s batch of size                6.4MB with   50000 of  250000 words
   |      |     3.83s batch of size                6.4MB with   50000 of  300000 words
   |      |     4.38

Now you have a file `~/Downloads/peshitta.mql` of 61 MB.
You can import it into an Emdros database by saying:

    cd ~/Downloads
    rm peshitta
    mql -b 3 < peshitta.mql
    
The result is an SQLite3 database `peshitta` in the same directory (24 MB).
You can run a query against it by creating a text file test.mql with this contents:

    select all objects where
    [book book=Gn
      [chapter chapter=1
        [verse verse=1]
      ]
    ]
    
And then say

    mql -b 3 -d peshitta test.mql
    
You will see raw query results: all words in Genesis 1:1.

It is not very pretty, and probably you should use a more visual Emdros tool to run those queries.
You see a lot of node numbers, but the good thing is, you can look those node numbers up in Text-Fabric.